In [13]:
from transformers import pipeline
classifer=pipeline('sentiment-analysis')

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [14]:
from transformers import AutoTokenizer ,TFAutoModelForSequenceClassification

In [15]:
model_name="cardiffnlp/twitter-roberta-base-sentiment-latest"
model=TFAutoModelForSequenceClassification.from_pretrained(model_name,from_pt=True)
tokenizer=AutoTokenizer.from_pretrained(model_name)
classifier=pipeline('sentiment-analysis',model=model_name,tokenizer=tokenizer)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaForSequenceClassification: ['roberta.embeddings.position_ids']
- This IS expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFRobertaForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.
Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment

In [16]:
import pandas as pd
file_path="NLP_Transformers_Using_huggingFace\Datasets\spamham.csv"
df=pd.read_csv(file_path)

In [17]:
X=list(df["sms"])
y=list(df["label"])
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.20,random_state=0)


In [18]:
#how to fine tune for preTrained Models
# 1) call pre trained models
#2) call for tokenizer for respect to pretrained models
from transformers import DistilBertTokenizerFast
tokenizer=DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [19]:
train_encodings=tokenizer(X_train,truncation=True,padding=True)
test_encodings=tokenizer(X_test,truncation=True,padding=True)

In [20]:
#3) convert encoding into data set objects
import tensorflow as tf
train_dataset=tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    y_train
))
test_dataset=tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    y_test
))

In [21]:
train_dataset

<_TensorSliceDataset element_spec=({'input_ids': TensorSpec(shape=(238,), dtype=tf.int32, name=None), 'attention_mask': TensorSpec(shape=(238,), dtype=tf.int32, name=None)}, TensorSpec(shape=(), dtype=tf.int32, name=None))>

In [22]:
from transformers import TFDistilBertForSequenceClassification , TFTrainingArguments
training_args=TFTrainingArguments(
    output_dir='./results',
    num_train_epochs=2,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
)

In [ ]:
from transformers import TFTrainer
with training_args.strategy.scope():
  model=TFDistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")
  trainer=TFTrainer(
      model=model,
      args=training_args,
      train_dataset=train_dataset,
      eval_dataset=test_datset
  )

In [ ]:
trainer.train()